<a href="https://colab.research.google.com/github/AlinZohari/InformationExtraction/blob/main/GoogleColab_SimpleTrans_TuneQAmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 29.3 MB/s eta 0:00:00

In [5]:
import requests
import json

url = "https://raw.githubusercontent.com/AlinZohari/InformationExtraction/main/data/QA_model/train.json"
response = requests.get(url)
train = response.json()

In [6]:
url = "https://raw.githubusercontent.com/AlinZohari/InformationExtraction/main/data/QA_model/validation.json"
response = requests.get(url)
validation = response.json()

In [7]:
import torch
print(torch.cuda.is_available())


True


In [8]:
train

[{'context': 'In this Order and Authorization, we grant, to the extent set forth below, the request of Kuiper Systems LLC (Kuiper or Amazon) to provide satellite services.\n            Operating 3,372 satellites in 102 orbital planes at altitudes of 590 km, 610 km, and 630 km in a circular orbit.\n            At 590 km, 30 orbital planes with 28 satellites per plane for a total of 840 satellites at inclination of 33 degree.\n            At 610 km, 42 orbital planes with 36 satellites per plane for a total of 1512 satellites at inclination of 42 degree.\n            At 630 km, 30 orbital planes with 34 satellites per plane for a total of 1020 satellite at inclination of 51.9 degree.\n            The constellation are require to launch and operate 50 percent of its satellites no later than July 30, 2026, and must launch the remaining space stations necessary to complete its authorized service constellation, place them in their assigned orbits, and operate each of them in accordance with 

In [9]:
validation

[{'context': 'Release date: October 29, 1995 In this Order and Authorization, we grant, to the extent set forth below, the request of Ligado Networks LLC to provide Fixed Satellite Services (FSS). Operating 2320 satellites in 58 orbital planes in total at altitudes of 500, 600, 700 and 800 kilometers. At an altitude of 500 km, there are 15 orbital planes, each hosting 36 satellites, resulting in a total of 540 satellites at an inclination of 36.5 degrees. For the 600 km altitude, 23 orbital planes are present, with each plane containing 50 satellites, summing up to 1150 satellites at an inclination of 49 degrees. At the 700 km mark, 15 orbital planes are equipped with 27 satellites each, leading to a total of 405 satellites at a 51.9-degree inclination. Lastly, at 800 km, there are 5 orbital planes, and each has 45 satellites, amounting to a total of 225 satellites at an inclination of 59.3 degrees. The constellation are require to launch and operate 50 percent of its satellites no lat

In [10]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs


In [11]:

model_type="bert"
model_name= "bert-base-cased"

if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [12]:

### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [13]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Train the model
model.train_model(train, eval_data=validation)

add example index and unique id: 100%|██████████| 42/42 [00:00<00:00, 247764.79it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Running Epoch 0 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 3 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(5,
 {'global_step': [1, 2, 3, 4, 5],
  'correct': [0, 0, 0, 0, 0],
  'similar': [5, 0, 4, 4, 4],
  'incorrect': [6, 11, 7, 7, 7],
  'train_loss': [4.902437210083008,
   4.966564178466797,
   4.535853862762451,
   4.296131134033203,
   4.097842216491699],
  'eval_loss': [-0.09930419921875,
   -0.07354736328125,
   -0.06951904296875,
   -0.0743408203125,
   -0.07952880859375]})

In [15]:
# Evaluate the model
result, texts = model.eval_model(validation)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
url = "https://raw.githubusercontent.com/AlinZohari/InformationExtraction/main/data/authorize_doc/StarlinkGen2_FCC-22-91A1.txt"
response = requests.get(url)
# Ensure the request was successful
if response.status_code == 200:
    context = response.text
else:
    print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")


In [18]:
#Define your list of questions
questions = [
    {
        "question": "What's the name of the satellite constellation the company seeks to deploy or operate?",
        "id": "const_name"
    },
    {
        "question": "On which date was the document released?",
        "id": "date_release"
    },
    {
        "question": "By which date must the company launch and operate half of its satellites?",
        "id": "date_50"
    },
    {
        "question": "By which date is the company expected to have all its satellites operational?",
        "id": "date_100"
    },
    {
        "question": "How many satellites is the company authorized to deploy and operate for this constellation?",
        "id": "total_sat_const"
    },
    {
        "question": "At which authorized altitudes will the company deploy its satellites?",
        "id": "altitude"
    },
    {
        "question": "What are the authorized satellite inclinations within the corresponding altitudes?",
        "id": "inclination"
    },
    {
        "question": "How many orbital planes, corresponding to given altitudes and inclinations, has the company been authorized for?",
        "id": "number_orb_plane"
    },
    {
        "question": "How many satellites are allocated to each orbital plane?",
        "id": "total_sat_per_orb_plane"
    },
    {
        "question": "How many satellites, for each altitude and inclination, are there across all matching orbital planes?",
        "id": "total_sat_per_alt_incl"
    },
    {
        "question": "What is the satellite's expected operational lifetime in years?",
        "id": "operational_lifetime"
    }
]

In [19]:
to_predict = [
    {
        "context": context,
        "qas": questions
    }
]

In [22]:

answers, probabilities = model.predict(to_predict)

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
answers

[{'id': 'const_name',
  'answer': ['November 29, 2022 Released: December 1, 2022 By the Commission: TABLE OF CONTENTS Heading Paragraph # I. INTRODUCTION 1',
   'ORDER AND AUTHORIZATION Adopted: November 29, 2022 Released: December 1, 2022 By the Commission: TABLE OF CONTENTS Heading Paragraph # I. INTRODUCTION 1']},
 {'id': 'date_release',
  'answer': ['November 29, 2022 Released: December 1, 2022 By the Commission: TABLE OF CONTENTS Heading Paragraph # I. INTRODUCTION 1',
   'ORDER AND AUTHORIZATION Adopted: November 29, 2022 Released: December 1, 2022 By the Commission: TABLE OF CONTENTS Heading Paragraph # I. INTRODUCTION 1']},
 {'id': 'date_50',
  'answer': ['November 29, 2022 Released: December 1, 2022 By the Commission: TABLE OF CONTENTS Heading Paragraph # I. INTRODUCTION 1',
   'ORDER AND AUTHORIZATION Adopted: November 29, 2022 Released: December 1, 2022 By the Commission: TABLE OF CONTENTS Heading Paragraph # I. INTRODUCTION 1']},
 {'id': 'date_100',
  'answer': ['November 2